<a href="https://colab.research.google.com/github/tommyA8/wangchanBERTa-fined-tune-thaiqa_squad/blob/main/wangchanBERTa_fined_tune_thaiqa_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation and Importing library

In [ ]:
!unzip "/content/Hack 4 - Extraction Question Answering.zip"

Archive:  /content/Hack 4 - Extraction Question Answering.zip
   creating: context/
  inflating: context/1345136.txt     
  inflating: context/1345137.txt     
  inflating: context/1345138.txt     
  inflating: context/1345139.txt     
  inflating: context/1345140.txt     
  inflating: context/1345141.txt     
  inflating: context/1345142.txt     
  inflating: context/1345143.txt     
  inflating: context/1345144.txt     
  inflating: context/1345145.txt     
  inflating: context/1345146.txt     
  inflating: context/1345147.txt     
  inflating: context/1345148.txt     
  inflating: context/1345149.txt     
  inflating: context/1345150.txt     
  inflating: context/1345153.txt     
  inflating: context/1345154.txt     
  inflating: context/1345155.txt     
  inflating: context/1345156.txt     
  inflating: context/1345157.txt     
  inflating: context/1345158.txt     
  inflating: context/1345159.txt     
  inflating: context/1345160.txt     
  inflating: context/1345161.txt     
  in

In [ ]:
from pandas import DataFrame
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

```
{
    "question_id": question id
    "article_id": article id
    "context": article texts
    "question": question
    "answers":
        {
            "answer": answer text
            "answer_begin_position": answer beginning position
            "answer_end_position": answer exclusive upper bound position
        }
    ),
}
```

#load_dataset

# Data-preparation Function (thaiqa_dataset)

In [ ]:
train_df = pd.read_csv("train.csv")

In [ ]:
for index, row in train_df.iterrows():
    f = open("context/" + row['file'])
    train_df.at[index, 'context'] = f.read()

In [ ]:
train_df

,file,question,answer,answer_start,answer_end,context
0,1345136.txt,ใครเป็นผู้ออกตราสารหนี้ภาคเอกชน ไร้ใบตราสาร,ไม่มีการรวมศูนย์อย่างตราสารภาครัฐ แต่ส่วนใหญ่ด...,7716,7799,การซื้อขายตราสารหนี้ในไทย\n\n1.\tตลาดตราสารหนี...
1,1345195.txt,Penny Stock หุ้นเหรียญ มีลักษณะอย่างไร,*\tหุ้นที่มีราคาซื้อขายต่อหน่วยต่ำ (ในสหรัฐฯ ค...,3019,3138,ประเภทของธุรกิจและที่มาของเงินทุนในการทำธุรกิจ...
2,1345200.txt,ราคาต้นงวดคืออะไร,ราคาต้นทุนหรือราคาซื้อ,501,522,ผลตอบแทนและความเสี่ยงของกองทุนรวม\n\n2.\tผลตอบ...
3,1345179.txt,อัตราการใช้สิทธิของ NVDR คือเท่าไหร่,1 : 1 (NVRD : สินทรัพย์อ้างอิง) เสมอ,1012,1047,ตราสารที่เชื่อมโยงกับตราสารทุน\n\n5.\tใบสำคัญแ...
4,1345158.txt,หนังสือชี้ชวนส่วนสรุปข้อมูลสำคัญ (Fund Fact Sh...,เอกสารที่ให้ข้อมูลที่สำคัญกับผู้ลงทุน,1116,1152,ความรู้เบื้องต้นเกี่ยวกับกองทุนรวม\n\n1.\tลักษ...
...,...,...,...,...,...,...
405,1345160.txt,กองทุนรวมตราสารหนี้ระยะยาว (Long-Term Fixed-In...,ลงทุนในตราสารหนี้ที่มีอายุเฉลี่ยของตราสารหนี้ถ...,5707,5959,ความรู้เบื้องต้นเกี่ยวกับกองทุนรวม\n\n3.\tประเ...
406,1345146.txt,อัตราดอกเบี้ยนโยบายเปลี่ยนแปลง จะส่งผลอย่างไร,"อัตราดอกเบี้ยต่างๆในตลาดไม่ว่าจะเป็น เงินฝาก, ...",474,704,การประเมินมูลค่าและอัตราผลตอบแทนของตราสารหนี้\...
407,1345154.txt,การลงทุนที่เน้นลงทุนในหุ้นที่มีแนวโน้มการเติบโ...,การลงทุนแบบเน้นการเติบโต (Growth Investing),391,433,การประเมินมูลค่าหุ้นสามัญ\n\n5.\tกลยุทธ์การลงท...
408,1345160.txt,กองทุนที่ได้สิทธิประโยชน์ทางภาษี คืออะไร,กองทุนรวมที่ให้สิทธิประโยชน์ทางภาษีกับผู้ลงทุน...,14289,14351,ความรู้เบื้องต้นเกี่ยวกับกองทุนรวม\n\n3.\tประเ...


Data Structure ที่เราต้องจัดการ
question_id	article_id	context	question	answers

In [ ]:
import string
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
punct = [char for char in punct if not ( char == ','  or char == '"' or char == '.' or char == '/' or char == "'" ) ]  #ลองไม่ลบ " , " กับ " ' " และ " . " ว่าจะเอา "/" ด้วย
punct  = ''.join(punct)
punct

'!#$%&()*+-:;<=>?@[\\]^_`{|}~'

In [ ]:
# thaiqa_train_df['clean_context'], thaiqa_train_df['new_question'], thaiqa_train_df['new_answers'] = zip(*thaiqa_train_df.apply(
#     lambda x: prepare_data(x['context'], x['question'], x['answers']), axis=1 ))

# thaiqa_valid_df['clean_context'], thaiqa_valid_df['new_question'], thaiqa_valid_df['new_answers'] = zip(*thaiqa_valid_df.apply(
#     lambda x: prepare_data(x['context'], x['question'], x['answers']), axis=1 ))

Dataframe ที่เราต้องการประกอบด้วย question_id	article_id	context	question	answers	clean_context	new_question	new_answers

ตัวอื่นปกติ แต่ถ้า answers จะต้องเป็น {'answer': [""], 'answer_begin_position'

```
   "answers":
        {
            "answer": answer text
            "answer_begin_position": answer beginning position
            "answer_end_position": answer exclusive upper bound position
        }
```

In [ ]:
for index, row in train_df.iterrows():
    answers_dict = {'answer': [row['answer']], 'answer_begin_position': [row['answer_start']], 'answer_end_position': [row['answer_end']]}
    train_df.at[index, 'answers'] = answers_dict

In [ ]:
import re
import string

punct = string.punctuation
REPLACE_BY_SPLACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
punct = [char for char in punct if not ( char == ','  or char == '"' or char == '.' or char == '/' or char == "'" ) ]  #ลองไม่ลบ " , " กับ " ' " และ " . " ว่าจะเอา "/" ด้วย
punct  = ''.join(punct)

def clean_text(text):
  text = REPLACE_BY_SPLACE_RE.sub(' ', text)
  # text = BAD_SYMBOLS_RE.sub('', text)
  text = text.replace('x', '')
  text = text.replace(punct, '')
  text = text.replace('\t', '')
  text = text.replace('\n', '')
  return text



In [ ]:
train_df.to_csv("train_preprocess_df.csv", index=False)

In [ ]:
train_df.drop(columns=["answer_start", "answer", "answer_end"], inplace=True)

In [ ]:
train_df

,file,question,context,answers
0,1345136.txt,ใครเป็นผู้ออกตราสารหนี้ภาคเอกชน ไร้ใบตราสาร,การซื้อขายตราสารหนี้ในไทย\n\n1.\tตลาดตราสารหนี...,{'answer': ['ไม่มีการรวมศูนย์อย่างตราสารภาครัฐ...
1,1345195.txt,Penny Stock หุ้นเหรียญ มีลักษณะอย่างไร,ประเภทของธุรกิจและที่มาของเงินทุนในการทำธุรกิจ...,{'answer': ['* หุ้นที่มีราคาซื้อขายต่อหน่วยต่ำ...
2,1345200.txt,ราคาต้นงวดคืออะไร,ผลตอบแทนและความเสี่ยงของกองทุนรวม\n\n2.\tผลตอบ...,"{'answer': ['ราคาต้นทุนหรือราคาซื้อ'], 'answer..."
3,1345179.txt,อัตราการใช้สิทธิของ NVDR คือเท่าไหร่,ตราสารที่เชื่อมโยงกับตราสารทุน\n\n5.\tใบสำคัญแ...,{'answer': ['1 : 1 (NVRD : สินทรัพย์อ้างอิง) เ...
4,1345158.txt,หนังสือชี้ชวนส่วนสรุปข้อมูลสำคัญ (Fund Fact Sh...,ความรู้เบื้องต้นเกี่ยวกับกองทุนรวม\n\n1.\tลักษ...,{'answer': ['เอกสารที่ให้ข้อมูลที่สำคัญกับผู้ล...
...,...,...,...,...
405,1345160.txt,กองทุนรวมตราสารหนี้ระยะยาว (Long-Term Fixed-In...,ความรู้เบื้องต้นเกี่ยวกับกองทุนรวม\n\n3.\tประเ...,{'answer': ['ลงทุนในตราสารหนี้ที่มีอายุเฉลี่ยข...
406,1345146.txt,อัตราดอกเบี้ยนโยบายเปลี่ยนแปลง จะส่งผลอย่างไร,การประเมินมูลค่าและอัตราผลตอบแทนของตราสารหนี้\...,{'answer': ['อัตราดอกเบี้ยต่างๆในตลาดไม่ว่าจะเ...
407,1345154.txt,การลงทุนที่เน้นลงทุนในหุ้นที่มีแนวโน้มการเติบโ...,การประเมินมูลค่าหุ้นสามัญ\n\n5.\tกลยุทธ์การลงท...,{'answer': ['การลงทุนแบบเน้นการเติบโต (Growth ...
408,1345160.txt,กองทุนที่ได้สิทธิประโยชน์ทางภาษี คืออะไร,ความรู้เบื้องต้นเกี่ยวกับกองทุนรวม\n\n3.\tประเ...,{'answer': ['กองทุนรวมที่ให้สิทธิประโยชน์ทางภา...


In [ ]:
from sklearn.model_selection import train_test_split

thaiqa_train, thaiqa_valid = train_test_split(train_df, test_size=0.2)

#Convert Dataframe to Huggingface datasets

In [ ]:
import datasets

thaiqa_train = Dataset.from_dict(thaiqa_train)
thaiqa_valid = Dataset.from_dict(thaiqa_valid)
thaiqa_dataset = datasets.DatasetDict({"train": thaiqa_train,"validation": thaiqa_valid})

In [ ]:
thaiqa_dataset['train'], thaiqa_dataset['validation']

(Dataset({
     features: ['file', 'question', 'context', 'answers'],
     num_rows: 328
 }),
 Dataset({
     features: ['file', 'question', 'context', 'answers'],
     num_rows: 82
 }))

#Pre-processing (thaiqa_dataset)

**Ref. https://huggingface.co/course/chapter7/7?fw=pt#a-custom-training-loop**

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

In [ ]:
thaiqa_train

Dataset({
    features: ['file', 'question', 'context', 'answers'],
    num_rows: 328
})

In [ ]:
list_wrong_answer_fromthaiqa_dataset = []

max_length = 416
doc_stride = 32

def pre_processing_thaiqa(examples):
  wrong_answer_fromthaiqa_dataset = {} # stored wrong answer
  inputs = tokenizer(
      examples['question'],
      examples['context'],
      truncation="only_second", # Truncation only context
      max_length=max_length,
      stride=doc_stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
  )
  sample_mapping = inputs.pop("overflow_to_sample_mapping") # The N examples gave M features.
  offset_mapping = inputs.pop("offset_mapping")

  inputs["start_positions"] = []
  inputs["end_positions"] = []

  for i, offsets in enumerate(offset_mapping):

      sequence_ids = inputs.sequence_ids(i)

      token_context_start = 0
      while sequence_ids[token_context_start] != 1:
          token_context_start += 1

      token_context_end = len(inputs["input_ids"][i]) - 1
      while sequence_ids[token_context_end] != 1:
          token_context_end -= 1

      sample_index = sample_mapping[i]
      answers = examples["answers"][sample_index]
      start_char = answers["answer_begin_position"][0]
      end_char = answers["answer_end_position"][0]

      # if the answer is in the corresponding span of the context: (start_context <= answer position <= end_context)
      if (offsets[token_context_start][0] <= start_char and offsets[token_context_end][1] >= end_char):

          while offsets[token_context_start][0] <= start_char:
              token_context_start += 1
          start_positions = token_context_start - 1

          while offsets[token_context_end][1] >= end_char:
              token_context_end -= 1
          end_positions = token_context_end + 1

          try:
              # detect incorrect answer when compare with real answer from dataset cause's 'answer_begin_position' and 'answer_end_position' is wrong
              assert tokenizer.decode(inputs["input_ids"][i][start_positions:end_positions+1]) == answers["answer"][0]
              inputs["start_positions"].append(start_positions)
              inputs["end_positions"].append(end_positions)
          except:
              # print('incorrect',answers["answer"][0])
              wrong_answer_fromthaiqa_dataset['pre_process_ans'] = tokenizer.decode(inputs["input_ids"][i][start_positions:end_positions+1])
              wrong_answer_fromthaiqa_dataset['dataset_answer'] = answers['answer'][0]
              # wrong_answer_fromthaiqa_dataset['answers'] = examples['answers']
              list_wrong_answer_fromthaiqa_dataset.append(wrong_answer_fromthaiqa_dataset)
              wrong_answer_fromthaiqa_dataset = {}
              inputs["start_positions"].append(0)
              inputs["end_positions"].append(0)
      else:
          # if the answer is not in the corresponding span of the context
          inputs["start_positions"].append(0)
          inputs["end_positions"].append(0)

  return inputs

In [ ]:
example = pre_processing_thaiqa(thaiqa_dataset['train'][0:5])
print(len(example['start_positions']), len(example['input_ids']))
print(example['start_positions'])

10 10
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
tokenized_thaiqa_dataset = thaiqa_dataset.map(pre_processing_thaiqa, batched=True, remove_columns=thaiqa_dataset["train"].column_names)

Map:   0%|          | 0/328 [00:00<?, ? examples/s]

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

In [ ]:
len(thaiqa_dataset['train']), len(tokenized_thaiqa_dataset["train"])
#จำนวนเพิ่มขึ้นจากการ truncation ส่งผลให้มีฟีเจอร์เกิดขึ้นมาเพิ่ม

(328, 1931)

In [ ]:
len(thaiqa_dataset["validation"]), len(tokenized_thaiqa_dataset['validation'])

(82, 409)

**'answer_begin_position' and 'answer_end_position' are incorrect.**

In [ ]:
len(list_wrong_answer_fromthaiqa_dataset)

301

In [ ]:
worng_ans_df = DataFrame(list_wrong_answer_fromthaiqa_dataset)
worng_ans_df

,pre_process_ans,dataset_answer
0,<unk>entral <unk>ank <unk>ill พันธบัตรระยะสั้น...,Central Bank Bill พันธบัตรระยะสั้นที่ออกโดยธนา...
1,ตราสารหนี้บางประเภทอาจมีสิทธิพิเศษบางอย่างติดม...,ตราสารหนี้บางประเภทอาจมีสิทธิพิเศษบางอย่างติดม...
2,ความเสี่ยงจากโครงสร้างการเงินของบริษัท (โครงสร...,ความเสี่ยงจากโครงสร้างการเงินของบริษัท (โครงสร...
3,การลงทุนแบบเน้นการเติบโต (<unk>rowth <unk>nves...,การลงทุนแบบเน้นการเติบโต (Growth Investing)
4,* ให้ข้อมูลด้านเศรษฐกิจทั้งในและต่างประเทศ * ใ...,*\tให้ข้อมูลด้านเศรษฐกิจทั้งในและต่างประเทศ\n\...
...,...,...
296,1. ใบสําคัญแสดงสิทธิอนุพันธ์ (<unk>erivative <...,1.\tใบสำคัญแสดงสิทธิอนุพันธ์ (Derivative Warra...
297,* ให้ข้อมูลเกี่ยวกับตราสารทุนและตราสารหนี้ เช่...,*\tให้ข้อมูลเกี่ยวกับตราสารทุนและตราสารหนี้ เช...
298,* เมื่อมีการขายคืนอัตโนมัติ <unk> per <unk>nit...,*\tเมื่อมีการขายคืนอัตโนมัติ NAV per Unit จะไม...
299,ถ้าคาดว่าสินทรัพย์ใดจะให้ผลตอบแทนที่ดีในอนาคต ...,ถ้าคาดว่าสินทรัพย์ใดจะให้ผลตอบแทนที่ดีในอนาคต ...


# Fine-tuning

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator

model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) #using the same tokenizer of WangchanBERTa

model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of CamembertForQuestionAnswering were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 32
learning_rate = 2e-5

args = TrainingArguments(
    f"finetune_thaiqa_squad",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    warmup_ratio=0.2,
    weight_decay=0.01,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=args,
    data_collator = default_data_collator,
    train_dataset=tokenized_thaiqa_dataset['train'],
    eval_dataset=tokenized_thaiqa_dataset['validation'],
    tokenizer=tokenizer,
)

/tmp/ipython-input-25-3977843892.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: folk-sawit (folk-sawit-kmutt) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=305, training_loss=2.152964707671619, metrics={'train_runtime': 104.1136, 'train_samples_per_second': 92.735, 'train_steps_per_second': 2.929, 'total_flos': 2049791401357440.0, 'train_loss': 2.152964707671619, 'epoch': 5.0})

In [44]:
#----------------------------------------save model-------------
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/

trainer.save_model("/content/QA_finetune_wangchan")

#Using the fine-tuned model

In [45]:
from transformers import pipeline

# Replace this with your own checkpoint
model = "/content/QA_finetune_wangchan"
question_answerer = pipeline("question-answering", model=model)

Device set to use cuda:0


## Test set - Context with BM25

In [ ]:
!pip install rank_bm25

In [30]:
import os
import glob
import pandas as pd

from rank_bm25 import BM25Okapi

from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

CONTEXT_DIR = "./context/"  # path to folder of .txt files TEST_FILE = "./test.csv"    # path to test.csv TOP_K = 3                     # number of contexts to retrieve per question
TOP_K = 3
# 1. Load contexts

contexts = {}
doc_texts = []
doc_names = []
for path in glob.glob(os.path.join(CONTEXT_DIR, "*.txt")):
  name = os.path.basename(path)
  with open(path, "r", encoding="utf-8") as f:
    text = f.read()
    contexts[name] = text
    doc_texts.append(text)
    doc_names.append(name)

# 2a. Build BM25 index

tokenized_corpus = [doc.split() for doc in doc_texts]
bm25 = BM25Okapi(tokenized_corpus)

# 2b. Build dense embeddings

model = SentenceTransformer('intfloat/multilingual-e5-base')
doc_embeddings = model.encode(doc_texts, convert_to_numpy=True)

# 3. Read test questions

TEST_FILE = "/content/test.csv"

test_df = pd.read_csv(TEST_FILE)

# 4. Retrieval functions

def retrieve_bm25(question, top_k=TOP_K):
  tokens = question.split()
  scores = bm25.get_scores(tokens)
  top_indices = np.argsort(scores)[::-1][:top_k]
  return [(doc_names[i], contexts[doc_names[i]], scores[i]) for i in top_indices]

def retrieve_dense(question, top_k=TOP_K):
  q_emb = model.encode([question], convert_to_numpy=True)
  sims = cosine_similarity(q_emb, doc_embeddings)[0]
  top_indices = np.argsort(sims)[::-1][:top_k]
  return [(doc_names[i], contexts[doc_names[i]], float(sims[i])) for i in top_indices]

# 5. Inference loop: get top contexts for each question

results = []
for idx, row in test_df.iterrows():
    q = row['question'] # Choose either BM25 or Dense candidates = retrieve_dense(q)
    candidates = retrieve_dense(q)
    for filename, context, score in candidates:
    # Format for filename, context, score in candidates: results.append({ 'question': q, 'context_file': filename, 'score': score, 'context_snippet': context[:200].replace("\n", " ") + '...' })
      results.append({ 'question': q, 'context_file': filename, 'score': score, 'context_snippet': context[:200].replace("\n", " ") + '...' })




# Usage:

# - Run this script after placing your test.csv and context/ folder

# - It produces 'retrieved_contexts.csv' with columns: question, context_file, score, snippet

# Next: feed these (question, context_file, context_text) pairs into your QA model for answer extraction.


In [34]:
# 6. Convert to DataFrame

output_df = pd.DataFrame(results)

#Keep top_k per question

output_df = output_df.groupby('question').head(TOP_K)

result_df = output_df.groupby('question')[['score', 'context_snippet']].agg(lambda x: x.value_counts().index[0])
result_df = pd.DataFrame(result_df)

# 7. Save

# output_df.to_csv("retrieved_contexts.csv", index=False, encoding='utf-8-sig')

In [35]:
result_df

,score,context_snippet
question,,
Absolute Valuation คืออะไร,0.834644,การประเมินมูลค่าหุ้นสามัญ 2. วิธีการประเมินมู...
All-bond index คืออะไร,0.840085,การซื้อขายตราสารหนี้ในไทย 4.\tดัชนีตราสารหนี้...
Bellwether index คืออะไร,0.827049,การซื้อขายตราสารหนี้ในไทย 4.\tดัชนีตราสารหนี้...
Board-based bond index คืออะไร,0.848306,การซื้อขายตราสารหนี้ในไทย 4.\tดัชนีตราสารหนี้...
Corporate Action คืออะไร,0.829538,ประเภทของธุรกิจและที่มาของเงินทุนในการทำธุรกิจ...
...,...,...
แบบแสดงรายงานข้อมูลประจำปี (แบบ 56-1) กำหนดส่งภายในกี่วัน,0.840784,ตลาดหลักทรัพย์แห่งประเทศไทย 9.\tข้อมูลที่บริษ...
โครงการสะสมหุ้นของพนักงาน (Employee joint investment program : EJIP) คืออะไร,0.850209,ประเภทของธุรกิจและที่มาของเงินทุนในการทำธุรกิจ...
ใครคือผู้จัดอันดับของทุนรวมที่เป็นที่นิยมของไทย,0.804837,การซื้อขายตราสารหนี้ในไทย 3.\tผู้เกี่ยวข้องกั...


In [36]:
test_df = pd.read_csv("test.csv")

inference_df = pd.merge(test_df, result_df, on="question", how="outer")

In [43]:
inference_df = inference_df.sort_values(by="id")
inference_df

,id,question,score,context_snippet
160,1,ตลาดหลักทรัพย์ได้กำหนดราคาสูงสุดต่ำสุดของ Warr...,0.864725,ตลาดหลักทรัพย์แห่งประเทศไทย 4.\tหน่วยการซื้อข...
205,2,ลักษณะแนวทางในการระดมทุนมีอะไรบ้าง,0.832773,ประเภทของธุรกิจและที่มาของเงินทุนในการทำธุรกิจ...
19,3,Gross Price หรือ Dirty Price มีวิธีการจ่ายดอกเ...,0.886564,การประเมินมูลค่าและอัตราผลตอบแทนของตราสารหนี้ ...
201,4,รายได้จากการลงทุนต่อ (Reinvestment income) คือ...,0.849310,ความรู้พื้นฐานของตราสารหนี้ 4.\tผลตอบแทนจากกา...
164,5,ตัวอย่างลำดับเหตุการณ์ที่เกี่ยวข้องกับการจ่ายเ...,0.832660,ประเภทของธุรกิจและที่มาของเงินทุนในการทำธุรกิจ...
...,...,...,...,...
187,271,ผู้จัดอันดับกองทุนรวมที่ได้รับอนุญาตจาก ก.ล.ต....,0.851266,ความรู้เบื้องต้นเกี่ยวกับกองทุนรวม 2.\tผู้ที่...
227,272,สูตรการคำนวณ Cash Settlement คืออะไร,0.828965,การประเมินมูลค่าและอัตราผลตอบแทนของตราสารหนี้ ...
106,273,ข้อดีของการประเมินมูลค่าด้วย PB Ratio คืออะไร,0.879952,การประเมินมูลค่าหุ้นสามัญ 4.\tRelative Valuat...
71,274,การรวมหุ้น (Reverse Stock Split) คืออะไร,0.828197,ประเภทของธุรกิจและที่มาของเงินทุนในการทำธุรกิจ...


In [46]:
import os
from tqdm import tqdm

model = "/content/QA_finetune_wangchan"
question_answerer = pipeline("question-answering", model=model)

real_ans = []

for index, row in tqdm(inference_df.iterrows(), total=inference_df.shape[0]):
  current_question = row['question']
  context = row['context_snippet']
  ans = question_answerer(question=current_question, context=context)
  real_ans.append(ans)

Device set to use cuda:0
100%|██████████| 275/275 [00:03<00:00, 88.36it/s]


In [ ]:
# import os
# from tqdm import tqdm

# model = "/content/QA_finetune_wangchan"
# question_answerer = pipeline("question-answering", model=model)

# real_ans = []

# cnt = 0

# for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
#   score = 0
#   cur_ans = ""
#   current_question = row['question']
#   for file in os.listdir("/content/context"):
#     cur_text = open("/content/context/" + file)
#     context = cur_text.read()
#     ans = question_answerer(question=current_question, context=context)
#     if ans['score'] > score:
#       print("New Score!", ans['score'])
#       cur_ans = ans
#       score = ans['score']
#   real_ans.append(cur_ans)
#   cnt += 1
#   print("append", cnt)
# #

In [ ]:
# import os
# from tqdm import tqdm
# from concurrent.futures import ThreadPoolExecutor
# from transformers import pipeline

# #Preload
# context_dir = "/content/context"
# contexts = []
# for file in os.listdir(context_dir):
#     file_path = os.path.join(context_dir, file)
#     with open(file_path, 'r') as f:
#         contexts.append(f.read())

# model_path = "/content/drive/MyDrive/QA_finetune_wangchan"
# question_answerer = pipeline("question-answering", model=model_path)
# question_answerer.model.to('cuda')  # Move model to GPU

# def process_question(question):
#     best_score = 0
#     best_answer = ""
#     for context in contexts:
#         ans = question_answerer(question=question, context=context)
#         if ans['score'] > best_score:
#             best_score = ans['score']
#             best_answer = ans
#     return best_answer

# real_ans = []
# with ThreadPoolExecutor() as executor:
#     results = list(executor.map(process_question, test_df['question']))

# real_ans = results

Device set to use cuda:0


In [59]:
sub_df = pd.read_csv("/content/sample_submission.csv")

for index, row in sub_df.iterrows():
  sub_df.at[index, 'answer'] = real_ans[index]['answer']
# sub_df['answer'] = real_ans['answer']

In [62]:
import re
import string

punct = string.punctuation
REPLACE_BY_SPLACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
punct = [char for char in punct if not ( char == ','  or char == '"' or char == '.' or char == '/' or char == "'" ) ]  #ลองไม่ลบ " , " กับ " ' " และ " . " ว่าจะเอา "/" ด้วย
punct  = ''.join(punct)

def clean_text(text):
  text = REPLACE_BY_SPLACE_RE.sub(' ', text)
  # text = BAD_SYMBOLS_RE.sub('', text)
  text = text.replace('x', '')
  text = text.replace(punct, '')
  text = text.replace('\t', '')
  text = text.replace('\n', '')
  return text

sub_df['answer'] = sub_df['answer'].apply(clean_text)

sub_df['answer'] = sub_df['answer'].str.replace('\d+','')

In [63]:
sub_df

,id,answer
0,1,หน่วยการซื้อขายในตลาดหลักทรัพย์แห่งประเทศไทยที...
1,2,5 ประเภท
2,3,การประเมินมูลค่าและอัตราผลตอบแทนของตราสารหนี้
3,4,ดอกเบี้ยที่ผู้ลงทุนได้รับ
4,5,4.1ผลตอบแทนของตราสารทุน
...,...,...
270,271,3 กลุ่มใหญ่ ดังนี้ 2.1กลุ่มผู้ประกอบการและผู้...
271,272,อัตราผลตอบแทนของตราสารหนี้มีหลายรูปแบบ ซึ่งแต...
272,273,การประเมินมูลค่าหุ้นสามัญ
273,274,การกระทำขององค์กรที่ทำให้เกิดการเปลี่ยนแปลงที...


In [ ]:
sub_df =

In [50]:
sub_df.head()

,id,answer
0,1,\tหน่วยการซื้อขายในตลาดหลักทรัพย์แห่งประเทศไทย...
1,2,5 ประเภท
2,3,การประเมินมูลค่าและอัตราผลตอบแทนของตราสารหนี้
3,4,ดอกเบี้ยที่ผู้ลงทุนได้รับ
4,5,4.1\tผลตอบแทนของตราสารทุน


In [49]:
sub_df.to_csv("bm25_dense_wangchan.csv", index=False)